Precipitation

Temperature

Boundary Layer Height

Wind

In [ ]:
#Calculate wind speed and direction based on u&v component

import xarray as xr
import numpy as np
import os
from glob import glob

# Define folders
input_folder = r"E:\IPMA\ERA5\UV_wind\1raw_year_1979_2024"
output_folder = r"E:\IPMA\ERA5\UV_wind\2wind_speed_direction"
os.makedirs(output_folder, exist_ok=True)

# Find all relevant NetCDF files
nc_files = sorted(glob(os.path.join(input_folder, "ERA5_hourly_uv_*.nc")))

for file_path in nc_files:
    print(f"Processing {os.path.basename(file_path)}")

    # Open dataset
    ds = xr.open_dataset(file_path)

    # Calculate wind speed
    wind_speed = np.sqrt(ds['u10']**2 + ds['v10']**2)

    # Calculate wind direction (degrees, meteorological convention)
    wind_dir = (180 + np.degrees(np.arctan2(ds['u10'], ds['v10']))) % 360

    # Add to dataset
    ds = ds.assign(wind_speed=wind_speed, wind_direction=wind_dir)

    # Add metadata
    ds['wind_speed'].attrs['units'] = 'm/s'
    ds['wind_speed'].attrs['description'] = '10m wind speed calculated from u10 and v10'
    ds['wind_direction'].attrs['units'] = 'degrees'
    ds['wind_direction'].attrs['description'] = 'Wind direction (from which wind blows, 0°=North, clockwise)'

    # Create output filename, e.g., ERA5_hourly_wind_1979.nc
    year_str = os.path.basename(file_path).split('_')[-1].split('.')[0]
    out_filename = f"ERA5_hourly_wind_{year_str}.nc"
    out_path = os.path.join(output_folder, out_filename)

    # Save only the wind_speed and wind_direction variables (optional)
    ds[['wind_speed', 'wind_direction']].to_netcdf(out_path)

    ds.close()

print("✅ All files processed and saved.")
